# Remote Objects

- distributed shared-memory object store
- think: passing variables across the cluster
- Ray will do object spilling to disk if needed

In Ray, we can create and compute on objects. We refer to these objects as remote objects, and we use object refs to refer to them. Remote objects are stored in shared-memory object stores, and there is one object store per node in the cluster. In the cluster setting, we may not actually know which machine each object lives on.

An object ref is essentially a unique ID that can be used to refer to a remote object. If you’re familiar with futures, our object refs are conceptually similar.

Object refs can be created in multiple ways:
- by remote function calls
- by `ray.put()`

In [1]:
import ray
import logging
import time

# Start Ray. If you're connecting to an existing cluster, you would use ray.init(address=<cluster-address>) instead.
ray.init(
    num_cpus=4,
    num_gpus=1,
    resources={'Custom': 2},
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)
ray.cluster_resources()                    # get the cluster resources

{'memory': 18389100135.0,
 'CPU': 4.0,
 'node:172.27.178.185': 1.0,
 'GPU': 1.0,
 'Custom': 2.0,
 'object_store_memory': 9194550067.0}

## Single object refs

Simple `put()` / `get()`.

In [8]:
y = 1
obj_ref = ray.put(y)
result = ray.get(obj_ref)
print(f"{result=}")
assert result == y

result=1


## Multiple object refs

Parallel `put()` / `get()`.

In [9]:
result = ray.get([ray.put(i) for i in range(3)])
print(f"{result=}")
assert result == [0, 1, 2]

result=[0, 1, 2]


## Timing out

You can timeout to return early from a `get()` that's blocking for too long.

In [3]:
from ray.exceptions import GetTimeoutError

@ray.remote
def long_running_function():
    time.sleep(8)

obj_ref = long_running_function.remote()
try:
    ray.get(obj_ref, timeout=4)
except GetTimeoutError:
    print("`get` timed out.")

`get` timed out.


## Waiting without blocking

After launching a number of tasks, you may want to know which ones have finisihed executing.

In [7]:
@ray.remote
def long_running_function():

    duration = 1 + 10 * ray.random_uniform.remote()
    time.sleep(duration)
    value = ray.random_uniform.remote() 
    return value


object_refs = [long_running_function.remote() for _ in range(10)]
ready_refs, remaining_refs = ray.wait(
    object_refs, num_returns=len(object_refs))
print(f"{ready_refs=}")
print(f"{remaining_refs=}")


ready_refs=[ObjectRef(34c9c2094e42fdbfffffffffffffffffffffffff0100000001000000), ObjectRef(747754f46b61f47dffffffffffffffffffffffff0100000001000000), ObjectRef(e11fe2800445c79affffffffffffffffffffffff0100000001000000), ObjectRef(139e431dd460af76ffffffffffffffffffffffff0100000001000000), ObjectRef(bbde8638d39a1245ffffffffffffffffffffffff0100000001000000), ObjectRef(44ed5e1383be6308ffffffffffffffffffffffff0100000001000000), ObjectRef(d56d800cbde6ca14ffffffffffffffffffffffff0100000001000000), ObjectRef(38db0ab51c6b6cfbffffffffffffffffffffffff0100000001000000), ObjectRef(4e2ab276f14c37c2ffffffffffffffffffffffff0100000001000000), ObjectRef(c96088b12950798bffffffffffffffffffffffff0100000001000000)]
remaining_refs=[]


## References

- [Ray Core Walkthrough: Objects in Ray](https://docs.ray.io/en/latest/walkthrough.html#objects-in-ray)